**Trabalho Final da disciplina Processamento de Dados em Larga Escala**

**Discentes:**

Valéria Cristina A. R. de Figueredo -- vcarf@cesar.school

Manuela de Lacerda Bezerra Carvalho -- mlbc@cesar.school

**Docente:** Anderson Neves

# **1. BIBLIOTECA**

In [ ]:
!pip install kagglehub kaggle

In [ ]:
import os
import pandas as pd
import kagglehub
import pyspark.sql.functions as F
from pyspark.sql import SparkSession


In [ ]:
!pip install --upgrade pyspark

# **2. "IMPORTAÇÃO DOS DADOS DO KAGGLE"**

In [ ]:
# Função para garantir que a autenticação Kaggle foi configurada
def check_kaggle_auth():
    if not os.path.exists(os.path.expanduser("~/.kaggle/kaggle.json")):
        raise FileNotFoundError("Arquivo de autenticação do Kaggle não encontrado. Por favor, coloque o arquivo 'kaggle.json' em ~/.kaggle/")

In [ ]:
# Baixar a versão mais recente do dataset
path = kagglehub.dataset_download("mkechinov/ecommerce-behavior-data-from-multi-category-store")

print("Caminho para os arquivos do dataset:", path)

100%|██████████| 4.29G/4.29G [00:47<00:00, 96.1MB/s]

Extracting files...


Caminho para os arquivos do dataset: /root/.cache/kagglehub/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store/versions/8


In [ ]:
# Caminho para os arquivos
dataset_path = '/root/.cache/kagglehub/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store/versions/8'

# Listando os arquivos no diretório
files = os.listdir(dataset_path)
print("Arquivos encontrados no diretório:", files)

Arquivos encontrados no diretório: ['2019-Nov.csv', '2019-Oct.csv']


In [ ]:
#Tamanho do arquivo '2019-Oct.csv'
# Caminho do arquivo
caminho_arquivo ='/root/.cache/kagglehub/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store/versions/8/2019-Oct.csv'

# Tamanho do arquivo em bytes
tamanho_bytes = os.path.getsize(caminho_arquivo)

# Converter bytes para GB
tamanho_gb = tamanho_bytes / (1024 ** 3)

print(f'Tamanho do arquivo: {tamanho_gb:.2f} GB')

Tamanho do arquivo: 5.28 GB


# **3. DICIONÁRIO DOS DADOS**



O dataset "eCommerce behavior data from multi category store" possui dados de comportamento de eCommerce de loja de múltiplas categorias.

Há dados de comportamento de 7 meses (de outubro de 2019 a abril de 2020) de uma grande loja online. No kaggle, está disponível para download direto apenas os meses de outubro e novembro de 2019.

Cada linha do arquivo representa um evento. Todos os eventos estão relacionados a produtos e usuários. Cada evento é uma relação de muitos para muitos entre produtos e usuários.

Os Dados foram coletados pelo projeto Open CDP.

As colunas e seus significados são os seguintes:



**Propriedade**          -                 **Descrição**

event_time	             - Hora em que o evento ocorreu (em UTC).

event_type	             - Tipo de evento (purchase, view ou cart).

product_id	             - ID de um produto.

category_id	             - ID da categoria do produto.

category_code	           - Taxonomia da categoria do produto (identificação da categoria).

brand	                   - Nome da marca em letras minúsculas.

price	                   - Preço do produto em formato float.

user_id	                 - ID permanente do usuário.

user_session             - ID temporário da sessão do usuário. É o mesmo para cada sessão do usuário. É alterado sempre que o usuário retorna à loja online após uma longa pausa.


[LINK DO DATASET ](https://www.kaggle.com/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store?select=2019-Nov.csv)

É importante ressaltar que o arquivo que será utilizado é o '2019-Oct.csv', com tamanho 5.67 GB.



# **4. AMBIENTE SPARK E BREVE ANÁLISE EXPLORATÓRIA**

In [ ]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '\
    --driver-memory 4G \
    --executor-memory 4G \
    pyspark-shell'

In [ ]:
# Criando um cluster local com 1 executor e a quantidade de threads igual a quantidade de cores de CPU disponíveis

spark = SparkSession.builder\
    .master("local[*]")\
    .config("spark.driver.bindAddress", "127.0.0.1")\
    .getOrCreate()
spark

In [ ]:
# Caminho do arquivo CSV (2019-Oct.csv)
csv_file_path = '/root/.cache/kagglehub/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store/versions/8/2019-Oct.csv'
# Ler o arquivo CSV
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

# Mostrar as primeiras linhas do DataFrame
df.show()

+-------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code|   brand|  price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|2019-10-01 00:00:00|      view|  44600062|2103807459595387724|                NULL|shiseido|  35.79|541312140|72d76fde-8bb3-4e0...|
|2019-10-01 00:00:00|      view|   3900821|2053013552326770905|appliances.enviro...|    aqua|   33.2|554748717|9333dfbd-b87a-470...|
|2019-10-01 00:00:01|      view|  17200506|2053013559792632471|furniture.living_...|    NULL|  543.1|519107250|566511c2-e2e3-422...|
|2019-10-01 00:00:01|      view|   1307067|2053013558920217191|  computers.notebook|  lenovo| 251.74|550050854|7c90fc70-0e80-459...|
|2019-10-01 00:00:04|      view|   1004237|2053013555631882655|electr

In [ ]:
df.printSchema()  # Verificar o schema do DataFrame

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



In [ ]:
df.columns

['event_time',
 'event_type',
 'product_id',
 'category_id',
 'category_code',
 'brand',
 'price',
 'user_id',
 'user_session']

In [ ]:
# Obter os valores distintos da coluna 'event_type'
df.select('event_type').distinct().show()

+----------+
|event_type|
+----------+
|  purchase|
|      view|
|      cart|
+----------+



In [ ]:
# Contar o número de valores nulos na coluna 'event_type'
df.filter(df['event_type'].isNull()).count()

0

# **5. QUESTÕES**

**1. Quais são as top 3 marcas (brand) mais populares, ou seja, com mais eventos na categoria computers.notebook (notebook.computador)? Ordene pela quantidade de eventos.**

In [ ]:
# Filtrar os dados pela categoria "computers.notebook"
df_filtered = df.filter(df['category_code'] == 'computers.notebook')

# Agrupar por marca (brand) e contar o número de eventos
df_brand_count = df_filtered.groupBy('brand').agg(F.count('*').alias('events_count'))

# Ordenar as marcas pela contagem de compras em ordem decrescente e pegar as top 3
df_top3_brands = df_brand_count.orderBy(F.col('events_count').desc()).limit(3)

# Exibir o resultado
df_top3_brands.show()

+------+------------+
| brand|events_count|
+------+------------+
|  acer|      332100|
|lenovo|      277025|
|    hp|      184966|
+------+------------+



**Resposta:**

As Top três marcas mais populares (mais eventos) na categoria computers notebook foram acer, lenovo e hp, como exposto acima.

**2.Qual dia em outubro ocorreram mais compras (purchase)?**

In [ ]:
# Converter a coluna 'event_time' para um formato de data
df = df.withColumn('event_date', F.to_date(df['event_time']))

# Filtrar apenas os eventos do tipo 'purchase' e para o mês de outubro
df_october_purchases = df.filter((df['event_type'] == 'purchase') &
                                 (F.month(df['event_date']) == 10)) # Finalidade: evitar erros, caso haja por equívoco no arquivo de outubro dados de outro mês.

# Agrupar por data e contar o número de compras por dia
df_daily_purchases = df_october_purchases.groupBy('event_date').agg(F.count('*').alias('purchase_count'))

# Ordenar pelos dias com mais compras
df_most_purchases_day = df_daily_purchases.orderBy(F.col('purchase_count').desc()).limit(1)

# Exibir o resultado
df_most_purchases_day.show()

+----------+--------------+
|event_date|purchase_count|
+----------+--------------+
|2019-10-16|         31394|
+----------+--------------+



**Resposta:**

O dia em outubro que ocorreu mais compras (purchase) foi 16.

**3. Qual horário as pessoas colocaram mais produtos da categoria electronics.video.tv no carrinho (cart)?**

In [ ]:
# Converter a coluna 'event_time' para o tipo timestamp
df = df.withColumn('time', F.col('event_time').cast('timestamp'))

# Filtrar os dados pela categoria 'electronics.video.tv' e tipo de evento 'cart'
df_filtered = df.filter((df['category_code'] == 'electronics.video.tv') & (df['event_type'] == 'cart'))

# Extrair a hora do evento (sem os minutos e segundos)
df_filtered = df_filtered.withColumn('hour', F.hour(df['time']))

# Agrupar por hora e contar o número de eventos 'cart' para cada hora
df_hourly_cart = df_filtered.groupBy('hour').agg(F.count('*').alias('cart_count'))

# Ordenar pelos horários com mais eventos 'cart' e pegar o horário com mais adições ao carrinho
df_most_cart_hour = df_hourly_cart.orderBy(F.col('cart_count').desc()).limit(1)

# Exibir o resultado
df_most_cart_hour.show()


+----+----------+
|hour|cart_count|
+----+----------+
|   9|      2596|
+----+----------+



**Resposta:**

As pessoas colocaram mais produtos da categoria electronics.video.tv no carrinho de compras (cart) às 9 horas da manhã.

**4. Qual a quantidade de cada tipo de evento para as 3 marcas com a maior receita?**

In [ ]:
#3 Marcas com a maior receita
df_brand = df.filter(df['event_type'] == 'purchase')\
    .groupBy('brand').agg(F.sum('price').alias('total_revenue'))\
    .orderBy(F.col('total_revenue').desc()).limit(3).cache()

df_brand.show()

+-------+-------------------+
|  brand|      total_revenue|
+-------+-------------------+
|  apple|1.112092688200001E8|
|samsung|4.640753261000007E7|
| xiaomi|  9194033.289999995|
+-------+-------------------+



In [ ]:
top_3_brands = df_brand.select(F.collect_list('brand')).first()[0]
top_3_brands

['apple', 'samsung', 'xiaomi']

In [ ]:
# Quantidade de cada tipo de evento (purchase, view, cart) para as 3 marcas com maior receita
df_5 = df.where(F.col('brand').isin(top_3_brands))\
      .groupBy('brand')\
      .pivot('event_type').agg(F.count("*")).cache()

df_5.show()

+-------+------+--------+-------+
|  brand|  cart|purchase|   view|
+-------+------+--------+-------+
|  apple|209084|  142873|3770597|
|samsung|303249|  172896|4806630|
| xiaomi|104497|   56616|2922650|
+-------+------+--------+-------+



**Resposta:**

A **Apple** teve **209084** eventos do tipo **cart**, **142873** do tipo **purchase** e **3770597** do tipo **view**.

A **Samsung** teve **303249** eventos do tipo **cart**, **172896** do tipo **purchase** e **4806630** do tipo **view**.

A **Xiomi** teve **104497** eventos do tipo **cart**, **56616** do tipo **purchase** e **2922650** do tipo **view**.